**Agradecimientos:** El contenido del siguiente jupyter notebook es la traducción al español del jupyter dispuesto en el siguiente [repositorio](https://github.com/wiqaaas/youtube/tree/master/Machine_Learning_from_Scratch/Search_Engine_Recommender) desarrollado por **Waqas Ahmed**.
Aquellas partes resaltadas en *cursivas* son aportes del profesor **Juan C. Correa**

# PageRank
Este jupyter supone que usted tiene conocimiento de autovectores y autovalores (*pero acá obviaremos eso por el momento*) y acá se muestra cómo se aplican con el algoritmo PageRank. Este jupyter consta de dos partes. La primera es una hoja de trabajo para que se ponga al día con el funcionamiento del algoritmo; aquí veremos un micro Internet con menos de 10 sitios web y veremos qué hace y qué puede salir mal. La segunda es una evaluación que pondrá a prueba su aplicación de la teoría propia a este problema escribiendo código y calculando el rango de página de una gran red que representa una subsección de Internet.

### 1- Page Rank en una red de Internet ficticia
##### Introducción

PageRank (desarrollado por Larry Page y Sergey Brin) revolucionó la búsqueda web al generar una lista clasificada de páginas web basada en la conectividad subyacente de la web. 

El algoritmo de PageRank se basa en un internauta aleatorio ideal que, al llegar a una página, pasa a la página siguiente haciendo clic en un enlace. 

El internauta tiene la misma probabilidad de hacer clic en cualquier enlace de la página y, cuando llega a una página sin enlaces, tiene la misma probabilidad de moverse a cualquier otra página escribiendo su URL. 

Además, el internauta puede optar ocasionalmente por escribir una URL aleatoria en lugar de seguir los enlaces de una página. 

El PageRank es el orden de clasificación de las páginas desde la página más probable hasta la menos probable que verá el internauta.


In [1]:
%pylab notebook
import numpy as np
import numpy.linalg as la
np.set_printoptions(suppress=True)

Populating the interactive namespace from numpy and matplotlib


### PageRank como un problema de algebra lineal

Imaginemos una pequeña red de Internet con apenas 6 páginas web, como las siguientes (**A**vocado, **B**ullseye, **C**atBabel, **D**romeda, **e**Tings, y **F**aceSpace). Cada página se vincula con algunas otras, y esta serie de conexiones forma una red como la siguiente

![A Micro-Internet](internet.png "A Micro-Internet")

El principio de diseño de PageRank es que los sitios web importantes estarán vinculados por sitios web importantes (*algo relacionado en teoría de redes con el concepto de Homofilia*). Este principio algo recursivo formará la base de nuestro pensamiento.

Imagina que tenemos 100 personas procrastinando en una Internet pequeña y cada uno ve un solo sitio web a la vez. Cada minuto, las personas siguen un enlace en su sitio web a otro sitio en esta Internet pequeña. Después de un tiempo, los sitios web a los que están más vinculados tendrán más personas visitándolos y, a la larga, cada minuto por cada persona que abandone un sitio web, entrará otro manteniendo constante el número total de personas en cada sitio web. El PageRank es simplemente la clasificación de los sitios web por la cantidad de personas que tiene cada página al final de este proceso.

Representamos el número de Pats en cada sitio web con el vector,

$$\mathbf{r} = \begin{bmatrix} r_A \\ r_B \\ r_C \\ r_D \\ r_E \\ r_F \end{bmatrix}$$
Y decimos que el número de personas en cada página web en el minuto $i+1$ está relacionado a aquellos en el minuto $i$ por la matriz de transformación siguiente:

$$ \mathbf{r}^{(i+1)} = L \,\mathbf{r}^{(i)}$$
with the matrix $L$ taking the form,
$$ L = \begin{bmatrix}
L_{A→A} & L_{B→A} & L_{C→A} & L_{D→A} & L_{E→A} & L_{F→A} \\
L_{A→B} & L_{B→B} & L_{C→B} & L_{D→B} & L_{E→B} & L_{F→B} \\
L_{A→C} & L_{B→C} & L_{C→C} & L_{D→C} & L_{E→C} & L_{F→C} \\
L_{A→D} & L_{B→D} & L_{C→D} & L_{D→D} & L_{E→D} & L_{F→D} \\
L_{A→E} & L_{B→E} & L_{C→E} & L_{D→E} & L_{E→E} & L_{F→E} \\
L_{A→F} & L_{B→F} & L_{C→F} & L_{D→F} & L_{E→F} & L_{F→F} \\
\end{bmatrix}
$$
donde las columnas representan la probabilidad de dejar una página para ir a cualquier otra, y sumarle 1. Las filas determinan cuán probable es que usted entre a una página a partir de cualquier otra, aunque estas probabilidades no necesitan sumar uno. El comportamiento a largo plazo de este sistema es cuando se cumple la siguiente igualdad $ \mathbf{r}^{(i+1)} = \mathbf{r}^{(i)}$, así que al quitar los sub-indices de esta fórmula podemos escribir,
$$ L \,\mathbf{r} = \mathbf{r}$$

lo cual es una ecuación de autovalor para la matriz $L$, con autovalor 1 (esto está garantizado por la estructura probabilística de dicha matriz).

In [ ]:
L = np.array([[0,   1/2, 1/3, 0, 0,   0 ],
              [1/3, 0,   0,   0, 1/2, 0 ],
              [1/3, 1/2, 0,   1, 0,   1/2 ],
              [1/3, 0,   1/3, 0, 1/2, 1/2 ],
              [0,   0,   0,   0, 0,   0 ],
              [0,   0,   1/3, 0, 0,   0 ]])

En principio, podríamos usar una biblioteca de álgebra lineal, como se muestra a continuación, para calcular los valores propios y los vectores. Y esto funcionaría para un sistema pequeño. Pero esto se vuelve inmanejable para sistemas grandes (*aquí entra la magia, la fama, y la reputación de lo que se llama Big Data*). Y dado que solo nos importa el vector propio principal (el que tiene el valor propio más grande, que será 1 en este caso), podemos usar el **método de iteración de potencia** que escalará mejor y es más rápido para sistemas grandes.

Utilice el siguiente código para echar un vistazo al PageRank de este micro Internet. (*Ahora sí toca entender intuitivamente qué es un autovalor y un autovector mirando este [video](https://youtu.be/YtgU1ozMgS0)* )

In [ ]:
eVals, eVecs = la.eig(L) # Obtenga los autovalores y autovectores
order = np.absolute(eVals).argsort()[::-1] # Ordenelos por su autovalores
eVals = eVals[order]
eVecs = eVecs[:,order]

r = eVecs[:, 0] # Defina r como el autovector principal
100 * np.real(r / np.sum(r)) # Haga que este autovector sume 1, y luego multipliquelo por 100 (las 100 personas)

Podemos ver en esta lista, la cantidad de personas que esperamos encontrar en cada sitio web después de mucho tiempo. Poniéndolos en orden de popularidad (según esta métrica), el PageRank de este micro Internet es:

CatBabel, Dromeda, Aguacate, FaceSpace, Bullseye, eTings

Volviendo al diagrama de nuestra pequeña internet, ¿es esto lo que hubiera esperado? Convénzase usted mismo de que, según la importancia aparente de las páginas, en términos de qué otras páginas se enlazan con ellas, esta es una clasificación sensata.

Intentemos ahora obtener el mismo resultado usando el método de potencia de iteración que se cubrió en este [video](https://youtu.be/a5zPyhQf7xw). Este método será mucho mejor para trabajar con sistemas grandes.

Primero, configuremos nuestro vector inicial, $\mathbf{r}^{(0)}$, de modo que tengamos las 100 personas distribuidas equitativamente en cada uno de nuestros 6 sitios web.

In [ ]:
r = 100 * np.ones(6) / 6 # Sets up this vector (6 entries of 1/6 × 100 each)
r # Shows it's value

Ahora, vamos a actualizar el vector al siguiente minuto con la matriz $L$.
Y corremos nuevamente en un loop lo anterior para ver si se estabiliza.

In [ ]:
for i in np.arange(100) : # Repeat 100 times
    r = L @ r
r

O mejor aún, podemos mantenerOr even better, we can keep running until we get to the required tolerance.

In [ ]:
r = 100 * np.ones(6) / 6 # Sets up this vector (6 entries of 1/6 × 100 each)
lastR = r
r = L @ r
i = 0
while la.norm(lastR - r) > 0.01 :
    lastR = r
    r = L @ r
    i += 1
print(str(i) + " iterations to convergence.")
r

See how the PageRank order is established fairly quickly, and the vector converges on the value we calculated earlier after a few tens of repeats.

Congratulations! You've just calculated your first PageRank!

#### Damping Parameter
The system we just studied converged fairly quickly to the correct answer.
Let's consider an extension to our micro-internet where things start to go wrong.

Say a new website is added to the micro-internet: *Geoff's* Website.
This website is linked to by *FaceSpace* and only links to itself.
![An Expanded Micro-Internet](internet2.png "An Expanded Micro-Internet")

Intuitively, only *FaceSpace*, which is in the bottom half of the page rank, links to this website amongst the two others it links to,
so we might expect *Geoff's* site to have a correspondingly low PageRank score.

Build the new $L$ matrix for the expanded micro-internet, and use Power-Iteration on the Procrastinating Pat vector.
See what happens…

In [ ]:
 # We'll call this one L2, to distinguish it from the previous L.
L2 = np.array([[0,   1/2, 1/3, 0, 0,   0, 0 ],
               [1/3, 0,   0,   0, 1/2, 0, 0 ],
               [1/3, 1/2, 0,   1, 0,   0, 0 ],
               [1/3, 0,   1/3, 0, 1/2, 0, 0 ],
               [0,   0,   0,   0, 0,   0, 0 ],
               [0,   0,   1/3, 0, 0,   1, 0 ],
               [0,   0,   0,   0, 0,   0, 1 ]])

In [ ]:
r = 100 * np.ones(7) / 7 # Sets up this vector (7 entries of 1/7 × 100 each)
lastR = r
r = L2 @ r
i = 0
while la.norm(lastR - r) > 0.01 :
    lastR = r
    r = L2 @ r
    i += 1
print(str(i) + " iterations to convergence.")
r

That's no good! *Geoff* seems to be taking all the traffic on the micro-internet, and somehow coming at the top of the PageRank.
This behaviour can be understood, because once a Pat get's to *Geoff's* Website, they can't leave, as all links head back to Geoff.

To combat this, we can add a small probability that the Procrastinating Pats don't follow any link on a webpage, but instead visit a website on the micro-internet at random.
We'll say the probability of them following a link is $d$ and the probability of choosing a random website is therefore $1-d$.
We can use a new matrix to work out where the Pat's visit each minute.
$$ M = d \, L + \frac{1-d}{n} \, J $$
where $J$ is an $n\times n$ matrix where every element is one.

If $d$ is one, we have the case we had previously, whereas if $d$ is zero, we will always visit a random webpage and therefore all webpages will be equally likely and equally ranked.
For this extension to work best, $1-d$ should be somewhat small - though we won't go into a discussion about exactly how small.

Let's retry this PageRank with this extension.

In [ ]:
d = 0.5 # Feel free to play with this parameter after running the code once.
M = d * L2 + (1-d)/7 * np.ones([7, 7]) # np.ones() is the J matrix, with ones for each entry.

In [ ]:
r = 100 * np.ones(7) / 7 # Sets up this vector (6 entries of 1/6 × 100 each)
lastR = r
r = M @ r
i = 0
while la.norm(lastR - r) > 0.01 :
    lastR = r
    r = M @ r
    i += 1
print(str(i) + " iterations to convergence.")
r

This is certainly better, the PageRank gives sensible numbers for the Procrastinating Pats that end up on each webpage.
This method still predicts Geoff has a high ranking webpage however.
This could be seen as a consequence of using a small network. We could also get around the problem by not counting self-links when producing the L matrix (an if a website has no outgoing links, make it link to all websites equally).
We won't look further down this route, as this is in the realm of improvements to PageRank, rather than eigenproblems.

You are now in a good position, having gained an understanding of PageRank, to produce your own code to calculate the PageRank of a website with thousands of entries.

Good Luck!

### 2. Sub Internet Page Ranking
In this assessment, you will be asked to produce a function that can calculate the PageRank for an arbitrarily large probability matrix.

In [ ]:
# Complete this function to provide the PageRank for an arbitrarily sized internet.
# I.e. the principal eigenvector of the damped system, using the power iteration method.
# (Normalisation doesn't matter here)
# The functions inputs are the linkMatrix, and d the damping parameter - as defined in this worksheet.
def pageRank(linkMatrix, d) :
    n = linkMatrix.shape[0]
    M = d * linkMatrix + (1-d)/n * np.ones([n, n])
    r = 100 * np.ones(n) / n # Sets up this vector (n entries of 1/n × 100 each)
    last = r
    r = M @ r
    while la.norm(last - r) > 0.01 :
        last = r
        r = M @ r
    return r

In [ ]:
def generate_internet(n) :
    c = np.full([n,n], np.arange(n))
    c = (abs(np.random.standard_cauchy([n,n])/2) > (np.abs(c - c.T) + 1)) + 0
    c = (c+1e-10) / np.sum((c+1e-10), axis=0)
    return c

In [ ]:
# Use the following function to generate internets of different sizes.
generate_internet(5)

In [ ]:
# Test your PageRank method against the built in "eig" method.
# You should see yours is a lot faster for large internets
L = generate_internet(100)

In [ ]:
pageRank(L, 1)

In [ ]:
# Do note, this is calculating the eigenvalues of the link matrix, L,
# without any damping. It may give different results that your pageRank function.
# If you wish, you could modify this cell to include damping.
eVals, eVecs = la.eig(L) # Gets the eigenvalues and vectors
order = np.absolute(eVals).argsort()[::-1] # Orders them by their eigenvalues
eVals = eVals[order]
eVecs = eVecs[:,order]

r = eVecs[:, 0]
100 * np.real(r / np.sum(r))

In [ ]:
# You may wish to view the PageRank graphically.
# This code will draw a bar chart, for each (numbered) website on the generated internet,
# The height of each bar will be the score in the PageRank.
# Run this code to see the PageRank for each internet you generate.
# Hopefully you should see what you might expect
# - there are a few clusters of important websites, but most on the internet are rubbish!
%pylab notebook
r = pageRank(generate_internet(100), 0.9)
plt.bar(arange(r.shape[0]), r);